In [3]:
import glob,os
import pandas as pd
import numpy as np
from datetime import datetime
import calendar
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [4]:
gastos=pd.read_csv("Gastos_diarios.csv")
precipitaciones=pd.read_csv("1PRECIPITACION.csv")
os.chdir("inputsGCM")
GCM=pd.read_excel("pr_day_ACCESS-ESM1-5_historical_r1i1p1f1_gn_19500101-19991231.xlsx")

/usr/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [3]:
print(gastos.head)

<bound method NDFrame.head of             Fecha  Gasto
0      1943-06-01    0.0
1      1943-06-02    0.0
2      1943-06-03    0.0
3      1943-06-04    0.0
4      1943-06-05    0.0
...           ...    ...
25777  2014-12-27    0.0
25778  2014-12-28    0.0
25779  2014-12-29    0.0
25780  2014-12-30    0.0
25781  2014-12-31    0.0

[25782 rows x 2 columns]>


In [5]:
def filtra_ts(ts,fechas):
    columna=fechas[3]
    i=datetime.strptime(fechas[0],fechas[2])
    f=datetime.strptime(fechas[1],fechas[2])
    ts[columna]=pd.to_datetime(ts[columna])
    filas=((ts[columna]>=i)&(ts[columna]<f)).values
    ts_study=ts.iloc[filas]
    ts_study=ts_study.set_index(columna)
    return ts_study
#list(GCM)
#list(gastos)
#list(precipitaciones)

In [6]:
# Fecha inicial, final, formato y columna.
fi="1951-01-01"
#ff="1989-10-01"
#ff="1992-01-01"
#ff="2000-01-01"
ff="2015-01-01"
fsty="%Y-%m-%d"

In [7]:
fechas=[fi,ff,fsty,"Fecha"]
q=filtra_ts(gastos,fechas)
#print(q.head)
p=filtra_ts(precipitaciones,fechas)
#print(p.head)
fechas=[fi,ff,fsty,list(GCM)[1]]
pGCM=filtra_ts(GCM,fechas)
pGCM.index.name="Fecha"
pGCM.index=pGCM.index.date
#pGCM

/tmp/ipykernel_192788/3609964137.py:5: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  ts[columna]=pd.to_datetime(ts[columna])


# Cuenta los datos faltantes.

In [8]:
# Añade los registros NaN
qdb=q.asfreq(freq="D")
pdb=p.asfreq(freq="D")
pGCMdb=pGCM.asfreq(freq="D")
qdb.to_csv("gastos_NaN.csv")

print(qdb.isna().sum())
dfaltan=pd.DataFrame(pdb.isna().sum())
dfaltan


Gasto    983
dtype: int64


,0
11020,3660
11023,4788
11025,224
11034,8349
11035,4558
11036,102
11040,76
11045,316
11054,9197
11055,7269


In [13]:
print(len(pdb.index))
print(len(qdb.index))
print(len(pGCMdb.index))

23376
23376
17897


### Simplifica los decimales

In [9]:
#pGCMdB=np.where()
#min=pGCMdb.between(0,0.05)
#pGCMdb[]=0.1
#pGCMdb[pGCMdb<=0.05]=0

## Correlaciones

In [10]:
puntosGCM=list(pGCMdb)
estaciones=list(pdb)
#print(pdb[estaciones[1]].head)
#print(pGCMdb[puntosGCM[1]].head)
database=pd.concat([pGCMdb,pdb],axis=1)
database.corr()



,Unnamed: 0,precipitaciones,precipitaciones.1,precipitaciones.2,precipitaciones.3,11020,11023,11025,11034,11035,...,11157,11162,14033,14070,14114,14369,16024,16065,16086,16117
Unnamed: 0,1.000000,0.016676,0.002748,0.007213,0.006374,-0.023173,-0.010948,-0.012723,-0.035461,-0.045674,...,0.013422,-0.013292,-0.015621,-0.030500,-0.008363,-0.039314,-0.015047,-0.046874,0.003882,-0.020337
precipitaciones,0.016676,1.000000,0.551183,0.767748,0.705416,0.101030,0.113016,0.107410,0.133308,0.114367,...,0.110969,0.112818,0.112507,0.143129,0.098078,0.117945,0.142634,0.123980,0.143652,0.129400
precipitaciones.1,0.002748,0.551183,1.000000,0.415163,0.711570,0.096187,0.092946,0.089523,0.122079,0.091939,...,0.096662,0.077904,0.092128,0.115942,0.083341,0.087616,0.111818,0.118813,0.120303,0.101069
precipitaciones.2,0.007213,0.767748,0.415163,1.000000,0.647859,0.104054,0.114148,0.108615,0.128605,0.118837,...,0.113037,0.120329,0.112888,0.132524,0.093864,0.133969,0.137037,0.134692,0.129413,0.127593
precipitaciones.3,0.006374,0.705416,0.711570,0.647859,1.000000,0.102306,0.106315,0.103930,0.131527,0.114922,...,0.109683,0.093620,0.107096,0.132911,0.095939,0.114556,0.137295,0.137253,0.128257,0.131067
11020,-0.023173,0.101030,0.096187,0.104054,0.102306,1.000000,0.562178,0.704026,0.311894,0.430248,...,0.326054,0.414579,0.550852,0.294945,0.436180,0.295225,0.294404,0.272262,0.332833,0.306512
11023,-0.010948,0.113016,0.092946,0.114148,0.106315,0.562178,1.000000,0.545005,0.356832,0.507619,...,0.398712,0.439877,0.482681,0.353250,0.406549,0.400689,0.355169,0.318209,0.380676,0.320869
11025,-0.012723,0.107410,0.089523,0.108615,0.103930,0.704026,0.545005,1.000000,0.340076,0.431884,...,0.332887,0.443011,0.606355,0.317183,0.458132,0.344160,0.313226,0.256398,0.365961,0.287873
11034,-0.035461,0.133308,0.122079,0.128605,0.131527,0.311894,0.356832,0.340076,1.000000,0.331331,...,0.294697,0.322921,0.358301,0.443145,0.280341,0.352800,0.432576,0.405130,0.538622,0.396618
11035,-0.045674,0.114367,0.091939,0.118837,0.114922,0.430248,0.507619,0.431884,0.331331,1.000000,...,0.361376,0.424230,0.392812,0.295299,0.321712,0.356286,0.290617,0.272384,0.357029,0.279082


In [11]:
print(pdb.head)
print(pGCMdb.head)

<bound method NDFrame.head of             11020  11023  11025  11034  11035  11036  11040  11045  11054  \
Fecha                                                                       
1951-01-01    NaN    NaN    0.0    NaN    NaN    0.0    0.6    0.0    NaN   
1951-01-02    NaN    NaN    0.0    NaN    NaN    0.0    0.0    0.0    NaN   
1951-01-03    NaN    NaN    0.0    NaN    NaN    0.0    0.0    0.0    NaN   
1951-01-04    NaN    NaN    0.0    NaN    NaN    0.0    0.0    0.0    NaN   
1951-01-05    NaN    NaN    0.0    NaN    NaN    0.0    0.0    0.0    NaN   
...           ...    ...    ...    ...    ...    ...    ...    ...    ...   
1999-12-27    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
1999-12-28    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
1999-12-29    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
1999-12-30    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
1999-12-31    0.0    0.0    0.0    0.0    0.0 

### Promedios

In [12]:
pGCMdb[puntosGCM[1]]

1951-01-01    0.245095
1951-01-02    0.087783
1951-01-03    0.741133
1951-01-04    1.077800
1951-01-05    0.275097
                ...   
1999-12-27    0.000005
1999-12-28    0.000199
1999-12-29    0.011928
1999-12-30    0.000000
1999-12-31    0.000567
Freq: D, Name: precipitaciones, Length: 17897, dtype: float64

In [15]:
def correlaciones(A,B):
    # Calcula las correlaciones entre estaciones y GCM.
    coeficientes=[]
    columnas=list(A)
    for i in columnas:
        C=pd.concat([A[i],B],axis=1)
        cor_punto=C.corr().iloc[0]
        coeficientes.append(cor_punto)
    cPearson=pd.DataFrame(coeficientes)
    return cPearson

def estadísticos(A):
    stat=[]
    stat.append(A.max())
    stat.append(A.mean())
    stat.append(A.min())
    stat.append(A.std())
    stat.append(A.var())
    s=pd.DataFrame(stat)
    s.index=["Máximo","Promedio","Mínimo","STD","Varianza"]
    return s

def bias(serie_estimada,serie):
    C=pd.concat([serie_estimada,serie],axis=1)
    C=C.dropna()
    dif =C.iloc[:,0]- C.iloc[:,1]
    total=sum(C.iloc[:,1])
    if total != 0:
        bbias=(sum(dif)/total)*100
    else:
        bbias=pd.NA
    return bbias

#A son las series estimadas y B es la serie observada.
def biasDF(A,B):
    filabias=A.apply(bias,serie=B,axis=0)
    return filabias

def rmse_na(A,B):
    C=pd.concat([A,B],axis=1)
    C=C.dropna()
    if len(C)==0:
        return pd.NA
    else:
        rmsecoef=mean_squared_error(C.iloc[:,0], C.iloc[:,1], squared=False)
    return rmsecoef

def rmseDF(matrizA,arregloB):
    filarmse=matrizA.apply(rmse_na,B=arregloB,axis=0)
    return filarmse

# Calcula el bias entre dos matrices A y B.
def stat_matrix(A,B,eval):
    columnas=list(B)
    statTab=pd.DataFrame()
    if eval == "bias":
        for i in columnas:
            biasA=biasDF(A,B[i])
            df=pd.DataFrame(biasA,columns=[[i]])
            statTab[i]=df
    elif eval == "rmse":
        for i in columnas:
            biasA=rmseDF(A,B[i])
            df=pd.DataFrame(biasA,columns=[[i]])
            statTab[i]=df
    return statTab



In [16]:
stObservados=estadísticos(pdb)
stObservados.to_csv("estadísticos_p_climatológicas.csv")
stObservados

,11020,11023,11025,11034,11035,11036,11040,11045,11054,11055,...,11157,11162,14033,14070,14114,14369,16024,16065,16086,16117
Máximo,109.000000,103.000000,115.000000,102.000000,90.000000,127.600000,126.000000,140.000000,100.600000,83.100000,...,107.000000,97.000000,108.000000,89.000000,110.300000,84.000000,87.800000,170.500000,75.000000,110.000000
Promedio,1.649376,1.712040,1.879820,1.984987,1.783760,1.940522,1.816773,2.130108,1.783024,1.716825,...,1.714459,1.694264,2.103821,2.254488,1.443007,2.183825,2.037386,2.096983,1.800826,1.833934
Mínimo,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
STD,5.685836,5.725150,6.192127,6.307360,6.038767,6.287786,6.088177,6.824581,6.059077,5.788921,...,6.474493,5.669649,6.714889,6.878200,5.463438,7.089558,6.093502,7.046264,5.718081,5.721154
Varianza,32.328734,32.777347,38.342432,39.782792,36.466705,39.536248,37.065903,46.574902,36.712415,33.511601,...,41.919058,32.144921,45.089737,47.309639,29.849151,50.261838,37.130770,49.649832,32.696446,32.731606


In [15]:
stGCM=estadísticos(pGCMdb)
stGCM.to_csv("estadísticos_p_GCM.csv")

In [16]:
corGCM=correlaciones(pGCMdb,pdb)
corGCM

,Unnamed: 0,11020,11023,11025,11034,11035,11036,11040,11045,11054,...,14114,14369,16024,16065,16086,16117,precipitaciones,precipitaciones.1,precipitaciones.2,precipitaciones.3
Unnamed: 0,1.0,-0.023173,-0.010948,-0.012723,-0.035461,-0.045674,-0.011108,-0.004051,-0.058126,-0.045591,...,-0.008363,-0.039314,-0.015047,-0.046874,0.003882,-0.020337,NaN,NaN,NaN,NaN
precipitaciones,NaN,0.101030,0.113016,0.107410,0.133308,0.114367,0.121093,0.105546,0.106361,0.118700,...,0.098078,0.117945,0.142634,0.123980,0.143652,0.129400,1.0,NaN,NaN,NaN
precipitaciones.1,NaN,0.096187,0.092946,0.089523,0.122079,0.091939,0.106147,0.094706,0.094934,0.088236,...,0.083341,0.087616,0.111818,0.118813,0.120303,0.101069,NaN,1.0,NaN,NaN
precipitaciones.2,NaN,0.104054,0.114148,0.108615,0.128605,0.118837,0.125905,0.104923,0.109552,0.126526,...,0.093864,0.133969,0.137037,0.134692,0.129413,0.127593,NaN,NaN,1.0,NaN
precipitaciones.3,NaN,0.102306,0.106315,0.103930,0.131527,0.114922,0.122880,0.100576,0.104583,0.116543,...,0.095939,0.114556,0.137295,0.137253,0.128257,0.131067,NaN,NaN,NaN,1.0


### Bias

In [45]:
biasST=stat_matrix(pGCMdb,pdb,"bias")
biasST.to_csv("bias.csv")



### RMSE

In [53]:
rmseST=stat_matrix(pGCMdb,pdb,"rmse")
rmseST.to_csv("rmse.csv")
rmseST

,11020,11023,11025,11034,11035,11036,11040,11045,11054,11055,...,11157,11162,14033,14070,14114,14369,16024,16065,16086,16117
Unnamed: 0,11874.230140,11542.097689,10670.015860,12792.714167,12102.312743,10636.203954,10645.338476,10655.212250,11399.219566,12921.959264,...,14757.265950,15113.247702,12068.928102,12253.209973,11622.404583,15053.336423,10581.932021,10204.638298,11337.785189,9474.435946
precipitaciones,8.701773,8.724730,8.948408,9.042683,9.039559,8.942908,8.946477,9.217741,8.947964,8.859839,...,9.438678,8.875780,9.317315,9.275643,8.708988,9.298922,8.810101,9.515882,8.809849,8.684084
precipitaciones.1,9.421130,9.469308,9.757655,9.781690,9.798809,9.760803,9.761247,9.963055,9.763034,9.652581,...,10.288840,9.736072,9.951102,9.997579,9.484302,10.217753,9.697164,10.191272,9.564394,9.631946
precipitaciones.2,8.374132,8.369461,8.695404,8.824565,8.711814,8.683959,8.696148,8.962735,8.616559,8.510043,...,9.160878,8.613028,9.067402,9.051985,8.397720,9.019697,8.611622,9.198845,8.493397,8.446580
precipitaciones.3,10.719098,10.703691,10.870927,10.935037,10.937242,10.822266,10.894106,11.011571,10.859621,10.847929,...,11.331808,10.914827,11.096236,11.054123,10.794541,11.203955,10.703581,11.132650,10.716146,10.617368


In [18]:
for file in glob.glob("*.xlsx"):
    GCM=pd.read_excel(file)
    fechas=[fi,ff,fsty,list(GCM)[1]]
    pGCM=filtra_ts(GCM,fechas)
    pGCM.index.name="Fecha"
    pGCM.index=pGCM.index.date
    pGCMdb=pGCM.asfreq(freq="D")
    corGCM=correlaciones(pGCMdb,pdb)
    corGCM.to_csv(file+"_corr.csv")


/usr/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [19]:
list(pGCMdb)

['Unnamed: 0',
 'precipitaciones',
 'precipitaciones.1',
 'precipitaciones.2',
 'precipitaciones.3']

In [20]:
sel=["11036","11040"]
muestra=pdb[sel]
muestrayGCM=muestra.join(pGCMdb)
muestrayGCM=muestrayGCM.dropna()
#X=muestrayGCM[["precipitaciones.1","precipitaciones.2","precipitaciones.3"]]
X=muestrayGCM[["precipitaciones.1","precipitaciones.3"]]
#X=muestrayGCM["precipitaciones.1"].values
Y=muestrayGCM["11040"].values
#Y=muestrayGCM[sel]
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, shuffle = False)

#Normalización
#from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

#sc = StandardScaler()
#scGCM = MinMaxScaler()
#Valores entre -1 y 1
scGCM = MinMaxScaler(feature_range=(-1, 1))
#X_train = X_train.reshape(-1,1)
#X_test = X_test.reshape(-1,1)
X_train = scGCM.fit_transform(X_train)
X_test = scGCM.transform(X_test)

#Inversa de las precipitaciones diarias normalizadas.
#X_train = 1-X_train
#X_test = 1-X_test

normXtrain=pd.DataFrame(X_train)
print(X_train.mean())

#scObs = MinMaxScaler()
scObs = MinMaxScaler(feature_range=(-1, 1))
y_train = y_train.reshape(-1,1)
y_test = y_test.reshape(-1,1)
y_train = scObs.fit_transform(y_train)
y_test = scObs.transform(y_test)

#Inversa de las precipitaciones diarias normalizadas.
#y_train = 1-y_train
#y_test = 1-y_test

normYtrain=pd.DataFrame(y_train)
print(y_train.mean())

normXtrain.to_csv("normXtrain.csv")
normYtrain.to_csv("normYtrain.csv")
#y_test = sc.fit(y_test)



-0.8462291723723265
-0.971940138484593


### Construir la red

In [21]:
import keras
from keras.models import Sequential
from keras.layers import Dense

2023-05-20 13:13:11.333038: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-20 13:13:12.036771: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-20 13:13:12.039946: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-20 13:13:15.015367: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [22]:
downscaler = Sequential()
downscaler.add(Dense(units = 60, activation = 'relu', input_dim = 2))
downscaler.add(Dense(units = 60, activation = 'sigmoid'))
#downscaler.add(Dense(units = 10, activation = 'relu'))
#downscaler.add(Dense(units = 30, activation = 'sigmoid'))
downscaler.add(Dense(units = 1, activation = 'linear'))
optimizador = keras.optimizers.SGD(learning_rate=0.01)
downscaler.compile(optimizer = optimizador, loss= keras.losses.MeanSquaredError(), metrics = ['accuracy'])
downscaler.fit(X_train, y_train, batch_size = 30, epochs = 20)



Epoch 1/20
473/473 [==============================] - 4s 5ms/step - loss: 0.0117 - accuracy: 7.0507e-05
Epoch 2/20
473/473 [==============================] - 2s 5ms/step - loss: 0.0086 - accuracy: 7.0507e-05
Epoch 3/20
473/473 [==============================] - 2s 4ms/step - loss: 0.0086 - accuracy: 7.0507e-05
Epoch 4/20
473/473 [==============================] - 2s 4ms/step - loss: 0.0086 - accuracy: 7.0507e-05
Epoch 5/20
473/473 [==============================] - 2s 4ms/step - loss: 0.0086 - accuracy: 7.0507e-05
Epoch 6/20
473/473 [==============================] - 2s 4ms/step - loss: 0.0085 - accuracy: 7.0507e-05
Epoch 7/20
473/473 [==============================] - 2s 4ms/step - loss: 0.0085 - accuracy: 7.0507e-05
Epoch 8/20
473/473 [==============================] - 2s 4ms/step - loss: 0.0085 - accuracy: 7.0507e-05
Epoch 9/20
473/473 [==============================] - 2s 4ms/step - loss: 0.0085 - accuracy: 7.0507e-05
Epoch 10/20
473/473 [==============================] - 2s 4ms/st

In [23]:
y_pred = downscaler.predict(X_test)
y_ann_output = scObs.inverse_transform(y_pred)
pd.DataFrame(y_pred).to_csv("prediccionesY.csv")
#pd.DataFrame(y_ann_output).to_csv("prediccionesY.csv")
#pd.DataFrame(X_test).to_csv("validX.csv")
pd.DataFrame(y_test).to_csv("validY.csv")

111/111 [==============================] - 1s 8ms/step
